In [1]:
# Speech emotion recognition on MSP-IMPROV dataset
# feature: egemaps
# Algorithm: as described in Bagus TA et al., "RNN-based dimenisional SER",
#                            ASJ Autumn 2019.
# 2019-12-24: Modified from ser_improv_egamaps, change input to HFS
#             Filter only natural interaction and improvisation data
# 2019-12-25: Updaate to adapt train/test data-label, change it accordingly
#             to be run in csl directory (although data in improv_loso/)
# 2020-05-26: update for SC paper

import numpy as np
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, \
                         Dropout, Flatten, LSTM, BatchNormalization, \
                         RNN, SimpleRNN, Activation
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
import random as rn
import tensorflow as tf
# from attention_helper import AttentionDecoder
#from read_csv import load_features


Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# loading file and label, choose the feature type first!
feat = np.load('../improv_loso/data/feat_hfs_mspin_train.npy')
feat_train = feat.reshape(feat.shape[0], 1, feat.shape[1])

feat_test = np.load('../improv_loso/data/feat_hfs_mspin_test.npy')
feat_test = feat_test.reshape(feat_test.shape[0], 1, feat_test.shape[1])

# load train and test ata
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_file = pd.read_csv(list_path_train, index_col=None)

vad_list = [list_file['v'], list_file['a'], list_file['d']]
vad_train = np.array(vad_list).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

vad_test = [list_test ['v'], list_test['a'], list_test['d']]
vad_test = np.array(vad_test).T

feat = np.concatenate([feat_train, feat_test], axis=0)
vad = np.concatenate([vad_train, vad_test])

# uncomment the following to remove silence feature (HFS1)
feat = feat[:,:, :-1]

# load test data
print('Feature shape: ', feat.shape)
print('Label shape: ', vad.shape)



Feature shape:  (7166, 1, 46)
Label shape:  (7166, 3)


In [4]:
## standardization
scaled_vad = True

if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [5]:
# API model, if use RNN, first two rnn layer must return_sequences=True
def api_model():
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='feat_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    #net_speech = LSTM(64, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(dropout_rate)(net_speech)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_speech) for name in target_names]

    model = Model(input_speech, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model2 = api_model()
#model2.summary()



In [6]:
split = 5733   # train, train+dev = 5744

In [7]:
# main function for for LOSO
def main(time):
    model_1 = api_model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(feat[4586:], batch_size=32)
    np.save('5733npy/speech_5733-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2580).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5733npy/speech_5733.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 4s 829us/step - loss: 1.8433 - v_loss: 0.8164 - a_loss: 0.4771 - d_loss: 0.5463 - v_ccc: 0.1827 - a_ccc: 0.5216 - d_ccc: 0.4524 - val_loss: 2.2124 - val_v_loss: 0.8627 - val_a_loss: 0.6496 - val_d_loss: 0.7002 - val_v_ccc: 0.1370 - val_a_ccc: 0.3508 - val_d_ccc: 0.2998
Epoch 2/50
4586/4586 [==============================] - 1s 318us/step - loss: 1.5251 - v_loss: 0.6584 - a_loss: 0.3870 - d_loss: 0.4754 - v_ccc: 0.3400 - a_ccc: 0.6120 - d_ccc: 0.5228 - val_loss: 1.6770 - val_v_loss: 0.6617 - val_a_loss: 0.4775 - val_d_loss: 0.5368 - val_v_ccc: 0.3372 - val_a_ccc: 0.5226 - val_d_ccc: 0.4631
Epoch 3/50
4586/4586 [==============================] - 2s 352us/step - loss: 1.4741 - v_loss: 0.6233 - a_loss: 0.3847 - d_loss: 0.4704 - v_ccc: 0.3797 - a_ccc: 0.6162 - d_ccc: 0.5300 - val_loss: 1.5350 - val_v_loss: 0

Epoch 4/50
4586/4586 [==============================] - 2s 356us/step - loss: 1.4473 - v_loss: 0.6004 - a_loss: 0.3815 - d_loss: 0.4623 - v_ccc: 0.3982 - a_ccc: 0.6176 - d_ccc: 0.5369 - val_loss: 1.5284 - val_v_loss: 0.6022 - val_a_loss: 0.4238 - val_d_loss: 0.5017 - val_v_ccc: 0.3967 - val_a_ccc: 0.5765 - val_d_ccc: 0.4984
Epoch 5/50
4586/4586 [==============================] - 2s 367us/step - loss: 1.4085 - v_loss: 0.5788 - a_loss: 0.3729 - d_loss: 0.4575 - v_ccc: 0.4212 - a_ccc: 0.6273 - d_ccc: 0.5430 - val_loss: 1.4827 - val_v_loss: 0.5788 - val_a_loss: 0.4082 - val_d_loss: 0.4947 - val_v_ccc: 0.4200 - val_a_ccc: 0.5921 - val_d_ccc: 0.5053
Epoch 6/50
4586/4586 [==============================] - 2s 385us/step - loss: 1.3152 - v_loss: 0.5151 - a_loss: 0.3580 - d_loss: 0.4400 - v_ccc: 0.4838 - a_ccc: 0.6411 - d_ccc: 0.5599 - val_loss: 1.4355 - val_v_loss: 0.5429 - val_a_loss: 0.3900 - val_d_loss: 0.5022 - val_v_ccc: 0.4564 - val_a_ccc: 0.6102 - val_d_ccc: 0.4980
Epoch 7/50
4586/4586 [

Epoch 22/50
1433/1433 [==============================] - 0s 107us/step
[0.2436741441488266, 0.5302373766899109, 0.3976839482784271]
2
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 4s 958us/step - loss: 1.7927 - v_loss: 0.7656 - a_loss: 0.4951 - d_loss: 0.5373 - v_ccc: 0.2368 - a_ccc: 0.5067 - d_ccc: 0.4637 - val_loss: 1.9622 - val_v_loss: 0.7618 - val_a_loss: 0.5672 - val_d_loss: 0.6327 - val_v_ccc: 0.2376 - val_a_ccc: 0.4331 - val_d_ccc: 0.3671
Epoch 2/50
4586/4586 [==============================] - 2s 385us/step - loss: 1.5374 - v_loss: 0.6571 - a_loss: 0.3975 - d_loss: 0.4821 - v_ccc: 0.3423 - a_ccc: 0.6024 - d_ccc: 0.5180 - val_loss: 1.6225 - val_v_loss: 0.6737 - val_a_loss: 0.4427 - val_d_loss: 0.5054 - val_v_ccc: 0.3253 - val_a_ccc: 0.5579 - val_d_ccc: 0.4944
Epoch 3/50
4586/4586 [==============================] - 2s 380us/step - loss: 1.4810 - v_loss: 0.6268 - a_loss: 0.3801 - d_loss: 0.4713 - v_ccc: 0.3722 - a_ccc: 0.618

1433/1433 [==============================] - 0s 114us/step
[0.20074595510959625, 0.5175100564956665, 0.3642618656158447]
3
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 5s 1ms/step - loss: 1.7938 - v_loss: 0.7834 - a_loss: 0.4715 - d_loss: 0.5393 - v_ccc: 0.2150 - a_ccc: 0.5288 - d_ccc: 0.4624 - val_loss: 1.9765 - val_v_loss: 0.8034 - val_a_loss: 0.5434 - val_d_loss: 0.6300 - val_v_ccc: 0.1963 - val_a_ccc: 0.4572 - val_d_ccc: 0.3700
Epoch 2/50
4586/4586 [==============================] - 2s 359us/step - loss: 1.5382 - v_loss: 0.6629 - a_loss: 0.3933 - d_loss: 0.4825 - v_ccc: 0.3390 - a_ccc: 0.6053 - d_ccc: 0.5175 - val_loss: 1.5517 - val_v_loss: 0.6501 - val_a_loss: 0.4125 - val_d_loss: 0.4882 - val_v_ccc: 0.3487 - val_a_ccc: 0.5878 - val_d_ccc: 0.5118
Epoch 3/50
4586/4586 [==============================] - 2s 392us/step - loss: 1.4761 - v_loss: 0.6260 - a_loss: 0.3816 - d_loss: 0.4670 - v_ccc: 0.3730 - a_ccc: 0.6186 - d_ccc: 0.

Epoch 3/50
4586/4586 [==============================] - 2s 335us/step - loss: 1.4702 - v_loss: 0.6158 - a_loss: 0.3820 - d_loss: 0.4715 - v_ccc: 0.3838 - a_ccc: 0.6177 - d_ccc: 0.5283 - val_loss: 1.5202 - val_v_loss: 0.6239 - val_a_loss: 0.4067 - val_d_loss: 0.4888 - val_v_ccc: 0.3750 - val_a_ccc: 0.5936 - val_d_ccc: 0.5112
Epoch 4/50
4586/4586 [==============================] - 2s 338us/step - loss: 1.3987 - v_loss: 0.5625 - a_loss: 0.3676 - d_loss: 0.4684 - v_ccc: 0.4379 - a_ccc: 0.6330 - d_ccc: 0.5303 - val_loss: 1.4346 - val_v_loss: 0.5612 - val_a_loss: 0.3896 - val_d_loss: 0.4834 - val_v_ccc: 0.4380 - val_a_ccc: 0.6108 - val_d_ccc: 0.5166
Epoch 5/50
4586/4586 [==============================] - 2s 354us/step - loss: 1.3046 - v_loss: 0.4980 - a_loss: 0.3578 - d_loss: 0.4455 - v_ccc: 0.5002 - a_ccc: 0.6415 - d_ccc: 0.5537 - val_loss: 1.4084 - val_v_loss: 0.5445 - val_a_loss: 0.3922 - val_d_loss: 0.4714 - val_v_ccc: 0.4549 - val_a_ccc: 0.6080 - val_d_ccc: 0.5286
Epoch 6/50
4586/4586 [

...

Epoch 19/50
4586/4586 [==============================] - 2s 410us/step - loss: 1.1382 - v_loss: 0.4153 - a_loss: 0.3236 - d_loss: 0.4010 - v_ccc: 0.5853 - a_ccc: 0.6777 - d_ccc: 0.5988 - val_loss: 1.3622 - val_v_loss: 0.5041 - val_a_loss: 0.3727 - val_d_loss: 0.4848 - val_v_ccc: 0.4950 - val_a_ccc: 0.6274 - val_d_ccc: 0.5154
Epoch 20/50
4586/4586 [==============================] - 2s 406us/step - loss: 1.1428 - v_loss: 0.4138 - a_loss: 0.3253 - d_loss: 0.4064 - v_ccc: 0.5862 - a_ccc: 0.6761 - d_ccc: 0.5948 - val_loss: 1.4084 - val_v_loss: 0.5190 - val_a_loss: 0.3857 - val_d_loss: 0.5033 - val_v_ccc: 0.4801 - val_a_ccc: 0.6144 - val_d_ccc: 0.4970
Epoch 21/50
4586/4586 [==============================] - 2s 385us/step - loss: 1.1265 - v_loss: 0.4083 - a_loss: 0.3099 - d_loss: 0.4064 - v_ccc: 0.5904 - a_ccc: 0.6896 - d_ccc: 0.5935 - val_loss: 1.3803 - val_v_loss: 0.5107 - val_a_loss: 0.3732 - val_d_loss: 0.4958 - val_v_ccc: 0.4883 - val_a_ccc: 0.6269 - val_d_ccc: 0.5044
Epoch 22/50
4586/45

Epoch 9/50
4586/4586 [==============================] - 2s 373us/step - loss: 1.2366 - v_loss: 0.4629 - a_loss: 0.3439 - d_loss: 0.4319 - v_ccc: 0.5380 - a_ccc: 0.6572 - d_ccc: 0.5682 - val_loss: 1.3840 - val_v_loss: 0.5167 - val_a_loss: 0.3789 - val_d_loss: 0.4878 - val_v_ccc: 0.4826 - val_a_ccc: 0.6212 - val_d_ccc: 0.5122
Epoch 10/50
4586/4586 [==============================] - 2s 377us/step - loss: 1.2264 - v_loss: 0.4544 - a_loss: 0.3400 - d_loss: 0.4328 - v_ccc: 0.5443 - a_ccc: 0.6599 - d_ccc: 0.5694 - val_loss: 1.3673 - val_v_loss: 0.5142 - val_a_loss: 0.3775 - val_d_loss: 0.4752 - val_v_ccc: 0.4852 - val_a_ccc: 0.6226 - val_d_ccc: 0.5249
Epoch 11/50
4586/4586 [==============================] - 2s 376us/step - loss: 1.2073 - v_loss: 0.4475 - a_loss: 0.3365 - d_loss: 0.4223 - v_ccc: 0.5522 - a_ccc: 0.6635 - d_ccc: 0.5770 - val_loss: 1.3661 - val_v_loss: 0.5052 - val_a_loss: 0.3766 - val_d_loss: 0.4837 - val_v_ccc: 0.4940 - val_a_ccc: 0.6236 - val_d_ccc: 0.5163
Epoch 12/50
4586/458

Epoch 5/50
4586/4586 [==============================] - 2s 400us/step - loss: 1.3700 - v_loss: 0.5444 - a_loss: 0.3649 - d_loss: 0.4595 - v_ccc: 0.4552 - a_ccc: 0.6342 - d_ccc: 0.5405 - val_loss: 1.4053 - val_v_loss: 0.5448 - val_a_loss: 0.3798 - val_d_loss: 0.4804 - val_v_ccc: 0.4543 - val_a_ccc: 0.6206 - val_d_ccc: 0.5197
Epoch 6/50
4586/4586 [==============================] - 2s 390us/step - loss: 1.2804 - v_loss: 0.4906 - a_loss: 0.3476 - d_loss: 0.4436 - v_ccc: 0.5102 - a_ccc: 0.6525 - d_ccc: 0.5568 - val_loss: 1.5036 - val_v_loss: 0.6297 - val_a_loss: 0.3827 - val_d_loss: 0.4910 - val_v_ccc: 0.3697 - val_a_ccc: 0.6176 - val_d_ccc: 0.5091
Epoch 7/50
4586/4586 [==============================] - 2s 380us/step - loss: 1.2758 - v_loss: 0.4852 - a_loss: 0.3541 - d_loss: 0.4401 - v_ccc: 0.5148 - a_ccc: 0.6493 - d_ccc: 0.5600 - val_loss: 1.3813 - val_v_loss: 0.5243 - val_a_loss: 0.3746 - val_d_loss: 0.4823 - val_v_ccc: 0.4751 - val_a_ccc: 0.6257 - val_d_ccc: 0.5179
Epoch 8/50
4586/4586 [

Epoch 7/50
4586/4586 [==============================] - 2s 405us/step - loss: 1.2836 - v_loss: 0.4758 - a_loss: 0.3596 - d_loss: 0.4507 - v_ccc: 0.5249 - a_ccc: 0.6413 - d_ccc: 0.5502 - val_loss: 1.4080 - val_v_loss: 0.5393 - val_a_loss: 0.3864 - val_d_loss: 0.4815 - val_v_ccc: 0.4597 - val_a_ccc: 0.6139 - val_d_ccc: 0.5184
Epoch 8/50
4586/4586 [==============================] - 2s 397us/step - loss: 1.2477 - v_loss: 0.4657 - a_loss: 0.3445 - d_loss: 0.4354 - v_ccc: 0.5333 - a_ccc: 0.6557 - d_ccc: 0.5633 - val_loss: 1.3974 - val_v_loss: 0.5243 - val_a_loss: 0.3880 - val_d_loss: 0.4844 - val_v_ccc: 0.4750 - val_a_ccc: 0.6122 - val_d_ccc: 0.5155
Epoch 9/50
4586/4586 [==============================] - 2s 404us/step - loss: 1.2475 - v_loss: 0.4682 - a_loss: 0.3477 - d_loss: 0.4385 - v_ccc: 0.5347 - a_ccc: 0.6537 - d_ccc: 0.5641 - val_loss: 1.3961 - val_v_loss: 0.5263 - val_a_loss: 0.3859 - val_d_loss: 0.4832 - val_v_ccc: 0.4730 - val_a_ccc: 0.6143 - val_d_ccc: 0.5167
Epoch 10/50
4586/4586 

Epoch 26/50
1433/1433 [==============================] - 0s 161us/step
[0.1779371052980423, 0.5274513959884644, 0.3716481924057007]
19


In [8]:
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
# hist = model2.fit(feat[:split], vad[:split].T.tolist(), batch_size=32,
#                   validation_split=0.2, epochs=50, verbose=1, shuffle=True,
#                   callbacks=[earlystop])

# # outs =loss, mse_v, mse_a, mse_d, ccc_v, ccc_a, ccc_d
# # test data start from 5744/split
# metrik2 = model2.evaluate(feat[split:], vad[split:].T.tolist())#, a[8000:], d[8000:]])
# print(metrik2[-3:])

# ## make prediction
# #feat_predict = feat[4596:]
# #vad_predict = vad[4596:]

# predict2 = model2.predict(feat[4596:], batch_size=32)
# np.save('speech_npy_loso',  np.array(predict2).reshape(3, 2570).T)